# : take a multiple pdf with text,image,table
1. fetch the data from pdf
2. at lesat there should be 200 pages
3. if chunking(use the sementic chunking technique) required do chunking and then embedding
4. store it inside the vector database(use any of them 1. mongodb 2. astradb 3. opensearch 4.milvus) ## i have not discuss then you need to explore
5. create a index with all three index machnism(Flat, HNSW, IVF) ## i have not discuss then you need to explore
6. create a retriever pipeline
7. check the retriever time(which one is fastet)
8. print the accuray score of every similarity search
9. perform the reranking either using BM25 or MMR ## i have not discuss then you need to explore
10. then write a prompt template
11. generte a oputput through llm
12. render that output over the DOCx ## i have not discuss then you need to explore
as a additional tip: you can follow rag playlist from my youtube

### Load the pdf with text , images and tables


In [4]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders.parsers import TesseractBlobParser


loader = PyMuPDFLoader(
    file_path = "/Users/ebythomas/Github_Projects/Multimodal-RAG/data/2507.13334v1.pdf",
    extract_images = True, 
    images_parser = TesseractBlobParser(), 
    extract_tables = "markdown"

)


In [5]:
docs=loader.load()

In [6]:
docs

[Document(metadata={'producer': 'pikepdf 8.15.1', 'creator': 'arXiv GenPDF (tex2pdf:)', 'creationdate': '', 'source': '/Users/ebythomas/Github_Projects/Multimodal-RAG/data/2507.13334v1.pdf', 'file_path': '/Users/ebythomas/Github_Projects/Multimodal-RAG/data/2507.13334v1.pdf', 'total_pages': 165, 'format': 'PDF 1.5', 'title': 'A Survey of Context Engineering for Large Language Models', 'author': 'Lingrui Mei; Jiayu Yao; Yuyao Ge; Yiwei Wang; Baolong Bi; Yujun Cai; Jiazhi Liu; Mingyu Li; Zhong-Zhi Li; Duzhen Zhang; Chenlin Zhou; Jiayi Mao; Tianze Xia; Jiafeng Guo; Shenghua Liu', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content='A Survey of Context Engineering for Large\nLanguage Models\nLingrui Mei1,6,†\nJiayu Yao1,6,†\nYuyao Ge1,6,†\nYiwei Wang2\nBaolong Bi1,6,†\nYujun Cai3\nJiazhi Liu1\nMingyu Li1\nZhong-Zhi Li6\nDuzhen Zhang6\nChenlin Zhou4\nJiayi Mao5\nTianze Xia6\nJiafeng Guo1,6,†\nShenghua Liu1,6,†,\n1 Institut

In [7]:
len(docs)

165

####  semantic Chunking and embedding

In [34]:
import os
from dotenv import  load_dotenv
load_dotenv()

True

In [36]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [37]:

from langchain_openai.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")



In [38]:
len(embeddings.embed_query("hi"))


3072

In [18]:
from langchain_experimental.text_splitter import SemanticChunker
text_splitter = SemanticChunker(OpenAIEmbeddings())

text_splitter = SemanticChunker(
    embeddings=embeddings,
    min_chunk_size =200
)


In [19]:
chunked_doc = text_splitter.split_documents(docs)

In [20]:
len(chunked_doc)

524

#### Vectorsdb : Zilliz(cloud)




In [42]:
ZILLIZ_CLOUD_URI = os.getenv("ZILLIZ_CLOUD_URI")
ZILLIZ_CLOUD_API_KEY = os.getenv("ZILLIZ_CLOUD_API_KEY")


creating index

In [57]:
from langchain_milvus import  Milvus

vectorstore_FLAT = Milvus(
    embedding_function=embeddings,
    connection_args={
        "uri": ZILLIZ_CLOUD_URI,
        "token": ZILLIZ_CLOUD_API_KEY,
        "db_name": "default"  # optional — use "default" unless you're using multiple databases
    },
    index_params={"index_type": "FLAT", "metric_type": "L2"},
    consistency_level="Strong",
    drop_old=True,  # set to True if seeking to drop the collection with that name if it exists
    collection_name="DATA_FLAT_INDEX",  # this is  index name
)

vectorstore_HNSW = Milvus(
    embedding_function=embeddings,
    connection_args={
        "uri": ZILLIZ_CLOUD_URI,
        "token": ZILLIZ_CLOUD_API_KEY,
        "db_name": "default"  # optional — use "default" unless you're using multiple databases
    },
    index_params={"index_type": "HNSW", "metric_type": "L2"},
    consistency_level="Strong",
    drop_old=False,  # set to True if seeking to drop the collection with that name if it exists
    collection_name="DATA_HNSW_INDEX",  # this is  index name
)


vectorstore_IVF = Milvus(
    embedding_function=embeddings,
    connection_args={
        "uri": ZILLIZ_CLOUD_URI,
        "token": ZILLIZ_CLOUD_API_KEY,
        "db_name": "default"  # optional — use "default" unless you're using multiple databases
    },
    index_params={"index_type": "IVF_FLAT", "metric_type": "L2"},
    consistency_level="Strong",
    drop_old=False,  # set to True if seeking to drop the collection with that name if it exists
    collection_name="DATA_IVF_INDEX",  # this is  index name
)



uploading data into vector database

In [48]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(chunked_doc))]

In [58]:
vectorstore_FLAT.add_documents(chunked_doc,ids=uuids)
vectorstore_IVF.add_documents(chunked_doc,ids=uuids)
vectorstore_HNSW.add_documents(chunked_doc,ids=uuids)

['e51aa6e7-eced-4582-bac5-96bb6bc714c5',
 '97844ef8-7c22-4602-9b20-656b767b7280',
 '3b5d1276-e844-4037-b1f9-85b78fb21f5c',
 'b7bb6b03-1322-469f-81c3-ee58f225a820',
 '3d9cbf5a-b295-4fc5-b76f-b9bdc60b1a24',
 '0b6e885e-ac10-4d5e-b530-1832aa9bf677',
 'deeec32a-05d0-4a37-b697-9a1d75444823',
 '60cb8ae3-66ac-46d4-997f-00ea1fe9adff',
 '0b90c69d-db54-4a46-8fc0-6f052f5e605f',
 'ddbc7901-c25d-4d40-93da-ce935f48afad',
 'edb7b300-922f-4a6b-97c6-e76c3fa2e675',
 'a243d02f-69da-437e-82e0-adcceb7e847b',
 '3a7b7409-5902-4893-9d23-2132dfe551fe',
 'a1b8ede7-55ed-4b1a-ad3f-6d24f503f823',
 '9c926aec-7311-4ab8-be6b-c5ad3ebd93f3',
 '77012593-ddf7-40f5-a1f9-d4b3fa3f9450',
 'f6628e1f-9d29-474d-8684-72a9f5a8066b',
 '9cd6b341-2a06-4a0b-bbf4-af2b6a3eec89',
 'eed98fac-d005-4fbc-a2b3-bc4fde1c60cd',
 '977e2f86-659b-4bb0-8031-a087bcfdb35c',
 '7d326c44-c9ee-415f-ba7e-825f1b17d94a',
 '43ca3a2e-8af8-4855-8b85-f1acd0cc6374',
 '4d86040a-de02-46b7-ab4c-af000dfd1b2b',
 '945695f7-ab47-431f-b1af-30e0ea15e7bb',
 '799c0aa9-9851-

#### Creation of retriever  for vectordb

In [61]:
retreiver_flat = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 10,             # total results to return
        "lambda_mult": 0.5   # 0 = max diversity, 1 = max relevance
    }
)
retreiver_HNSW = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 10,             # total results to return
        "lambda_mult": 0.5   # 0 = max diversity, 1 = max relevance
    }
)
retreiver_IVF = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 10,             # total results to return
        "lambda_mult": 0.5   # 0 = max diversity, 1 = max relevance
    }
)

In [62]:
import time

query = "performance of Large Language Models (LLMs)"

def benchmark_retriever(name, retriever):
    start = time.time()
    results = retriever.invoke(query)
    duration = time.time() - start
    print(f"{name} took {duration:.4f} seconds")
    print(results)

# Run benchmarks
docs_flat = benchmark_retriever("FLAT", retreiver_flat)
docs_hnsw = benchmark_retriever("HNSW", retreiver_HNSW)
docs_ivf = benchmark_retriever("IVF", retreiver_IVF)


FLAT took 2.5279 seconds
[Document(metadata={'title': 'A Survey of Context Engineering for Large Language Models', 'subject': '', 'creationdate': '', 'page': 128, 'trapped': '', 'source': '/Users/ebythomas/Github_Projects/Multimodal-RAG/data/2507.13334v1.pdf', 'file_path': '/Users/ebythomas/Github_Projects/Multimodal-RAG/data/2507.13334v1.pdf', 'producer': 'pikepdf 8.15.1', 'creationDate': '', 'creator': 'arXiv GenPDF (tex2pdf:)', 'modDate': '', 'author': 'Lingrui Mei; Jiayu Yao; Yuyao Ge; Yiwei Wang; Baolong Bi; Yujun Cai; Jiazhi Liu; Mingyu Li; Zhong-Zhi Li; Duzhen Zhang; Chenlin Zhou; Jiayi Mao; Tianze Xia; Jiafeng Guo; Shenghua Liu', 'moddate': '', 'format': 'PDF 1.5', 'keywords': '', 'total_pages': 165, 'pk': '67ef66a2-43fe-4120-a60c-f1b9711572c0'}, page_content='Taylor. The evolving landscape of llm- and vlm-\nintegrated reinforcement learning. arXiv preprint, 2025. [935] Lianlei Shan, Shixian Luo, Zezhou Zhu, Yu Yuan, and Yong Wu. Cognitive memory in large\nlanguage models, arXi

### RAG  pipeline

In [ ]:
#model
from langchain_google_genai import ChatGoogleGenerativeAI
model =ChatGoogleGenerativeAI(model='gemini-1.5-flash')

